In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import json
pd.options.mode.chained_assignment = None

In [2]:
a = []
# b = []
matrix = []
heatmapMatrix = []

locList = ["Grocery Stores, Supermarkets", "Bakeries", "Fast Food Restaurants", "Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances", "Drug Stores and Pharmacies", "Book Stores", "Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"]

for j in range(len(locList)):
    prob = 1
    a = []
    b = []
    for i in range(len(locList)):
        if i != len(locList)-1:
            num = 0.16 * (i+1) *  np.random.rand()
            indprob = num * prob
            a.append(indprob)
            heatmapMatrix.append([locList[i], locList[j], indprob])
            prob = prob - indprob
        else:
            a.append(prob)
            heatmapMatrix.append([locList[i], locList[j], prob])
    matrix.append(a)
    # heatmapMatrix.append(b)

# print(heatmapMatrix)
probMatrix = pd.DataFrame(matrix, columns = locList, index = locList)
heatmapProbMatrix = pd.DataFrame(heatmapMatrix, columns=["place1", "place2", "prob"])

In [3]:
probMatrix.head()

,"Grocery Stores, Supermarkets",Bakeries,Fast Food Restaurants,"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",Drug Stores and Pharmacies,Book Stores,"Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"
"Grocery Stores, Supermarkets",0.128857,0.184480,0.329368,0.081776,0.128079,0.065464,0.081976
Bakeries,0.078421,0.092792,0.183401,0.118357,0.414119,0.055350,0.057560
Fast Food Restaurants,0.143871,0.000144,0.171438,0.109893,0.358278,0.164314,0.052061
"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",0.063132,0.123797,0.176975,0.052349,0.251782,0.191324,0.140641
Drug Stores and Pharmacies,0.130499,0.225488,0.042638,0.086000,0.278594,0.096408,0.140374


In [4]:
heatmapProbMatrix.to_csv("heatmapProbMatrix.csv")

In [5]:
pd.Timestamp("2020-04-30 09:00:00") + pd.offsets.Minute(10)  

Timestamp('2020-04-30 09:10:00')

In [9]:
%%time
allRec = []
routeLenth = 8
start_time = pd.Timestamp("2020-04-30")
transaction_type_list = ["online", "offline"]
random_merchant_list = ["merchantA", "merchantB", "merchantC"]

# for j in range(routeLenth-1):
# #     rec = []
#     name = "people"+str(j)
pplCount = 0
for j in range(300): 
    for num, location in zip(range(len(locList)), locList):
        name = "people"+str(pplCount)
        station = location
        rec = []
        time = pd.Timestamp("2020-04-30 09:00:00") 
        rec.append(name)#name
        rec.append(station)#location
        rec.append(np.random.choice(random_merchant_list, 1)[0])
        rec.append(time) #time
        rec.append(np.random.randint(10, high=80)) #transaction val
        rec.append(np.random.choice(transaction_type_list, 1)[0]) #transaction type
        allRec.append(rec)
        pplCount += 1
        for i in range(routeLenth-1):
            rec = []
            time += pd.offsets.Minute(np.random.randint(1, high=200))  
            # print(station)
            probList = probMatrix.loc[station].to_list()
            station = np.random.choice(locList, 1, p=probList)[0]
            rec.append(name)#name
            rec.append(station)#mcc
            rec.append(np.random.choice(random_merchant_list, 1)[0])
            rec.append(time) #time
            rec.append(np.random.randint(10, high=80)) #transaction val
            rec.append(np.random.choice(transaction_type_list, 1)[0]) #transaction type
            
            allRec.append(rec)



CPU times: user 4.04 s, sys: 24 ms, total: 4.06 s
Wall time: 4.05 s


In [10]:
records = pd.DataFrame(allRec, columns = ["name", "mcc", "merchant", "time", "transaction_value", "transaction_type"])

In [11]:
records.head()

,name,mcc,merchant,time,transaction_value,transaction_type
0,people0,"Grocery Stores, Supermarkets",merchantB,2020-04-30 09:00:00,74,offline
1,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online
2,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline
3,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online
4,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online


In [12]:
len(records)

16800

In [14]:
%%time
records["name"] = records["name"].astype("category")
records["mcc"] = records["mcc"].astype("category")
nameList = records["name"].unique()
record_with_tinterval = pd.DataFrame([], columns = ["name", "mcc", "merchant", "time", "transaction_value", "transaction_type", "time_interval_to_next", "time_interval_from_last"])
#calculate time interval
for name in nameList:
    record_by_name = records[records["name"] == name]
    record_by_name["time_interval_to_next"] = -record_by_name["time"].diff(periods = -1)
    record_by_name["time_interval_from_last"] = record_by_name["time"].diff(periods = 1)
    record_by_name.replace(pd.Timedelta('nat'), pd.Timedelta(0), inplace=True)
    
    record_with_tinterval = record_with_tinterval.append(record_by_name)
#     print(record_by_name.head(2), record_with_tinterval.head())

record_with_tinterval.head()

CPU times: user 10.3 s, sys: 14.7 ms, total: 10.3 s
Wall time: 10.3 s


,name,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last
0,people0,"Grocery Stores, Supermarkets",merchantB,2020-04-30 09:00:00,74,offline,01:12:00,00:00:00
1,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00
2,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00
3,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00
4,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00


In [15]:
record_with_tinterval.to_csv("data_example_short_with_type.csv")

# Here is the main code

In [7]:
%%time
#read data and change data type
record_with_tinterval = pd.read_csv("data_example_short_with_type.csv", index_col = 0)

# record_with_tinterval["name"] = record_with_tinterval["name"].astype("category")
record_with_tinterval["mcc"] = record_with_tinterval["mcc"].astype("category")
record_with_tinterval["transaction_type"] = record_with_tinterval["transaction_type"].astype("category")
record_with_tinterval["time"] =  pd.to_datetime(record_with_tinterval["time"])
record_with_tinterval["time_interval_to_next"] =  pd.to_timedelta(record_with_tinterval["time_interval_to_next"])
record_with_tinterval["time_interval_from_last"] =  pd.to_timedelta(record_with_tinterval["time_interval_from_last"])


CPU times: user 123 ms, sys: 3.19 ms, total: 126 ms
Wall time: 125 ms


In [8]:
record_with_tinterval.dtypes

name                                object
mcc                               category
merchant                            object
time                        datetime64[ns]
transaction_value                    int64
transaction_type                  category
time_interval_to_next      timedelta64[ns]
time_interval_from_last    timedelta64[ns]
dtype: object

In [9]:
record_with_tinterval.tail()

,name,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last
16795,people2099,Book Stores,merchantC,2020-04-30 12:59:00,61,online,00:07:00,02:58:00
16796,people2099,Fast Food Restaurants,merchantB,2020-04-30 13:06:00,60,online,00:21:00,00:07:00
16797,people2099,Drug Stores and Pharmacies,merchantA,2020-04-30 13:27:00,25,online,02:54:00,00:21:00
16798,people2099,Bakeries,merchantA,2020-04-30 16:21:00,63,online,02:07:00,02:54:00
16799,people2099,Drug Stores and Pharmacies,merchantB,2020-04-30 18:28:00,21,online,00:00:00,02:07:00


In [10]:
#apply time filter: start time -> end time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
test_time_range_filtered = record_with_tinterval[(record_with_tinterval["time"] > startTime)
                      & (record_with_tinterval["time"] < endTime)]

In [11]:
%%time
#like SQL's row_numebr, give a rank as time

test_time_range_filtered["step"] = test_time_range_filtered.groupby(["name"])["time"].rank().astype(int)

CPU times: user 6.76 ms, sys: 1.17 ms, total: 7.93 ms
Wall time: 6.37 ms


In [12]:
test_time_range_filtered.head()

,name,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last,step
1,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00,1
2,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00,2
3,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00,3
4,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00,4
5,people0,Bakeries,merchantB,2020-04-30 14:19:00,70,offline,00:11:00,01:25:00,5


In [13]:
len(test_time_range_filtered)

9802

In [14]:
%%time
queryMCC = "Bakeries"
#get largest min "max first position" of the queried MCC in the route as the left alignment position
test_data_query = test_time_range_filtered.loc[test_time_range_filtered["mcc"] == queryMCC]
grouped_test_data_query = test_data_query.groupby("name")
left_alignment_pos = grouped_test_data_query["step"].min().max()
need_alignment = grouped_test_data_query["step"].min().dropna() < left_alignment_pos
alignment_offset = left_alignment_pos - grouped_test_data_query["step"].min().dropna()
len(need_alignment)

CPU times: user 9.57 ms, sys: 2.59 ms, total: 12.2 ms
Wall time: 11 ms


914

In [15]:
%%time
test_data_query_route = test_time_range_filtered[test_time_range_filtered["name"].isin(need_alignment.index)]
test_data_query_route.head()

CPU times: user 3.09 ms, sys: 575 µs, total: 3.66 ms
Wall time: 3.2 ms


,name,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last,step
1,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00,1
2,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00,2
3,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00,3
4,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00,4
5,people0,Bakeries,merchantB,2020-04-30 14:19:00,70,offline,00:11:00,01:25:00,5


In [16]:
test_data_query_route.shape

(4481, 9)

In [17]:
%%time
#move the order acoording to offset
test_data_query_route.set_index("name", inplace = True)
test_data_query_route["offset"] = alignment_offset
test_data_query_route["step"] = test_data_query_route["step"] + test_data_query_route["offset"]
test_data_query_route.head()

CPU times: user 3.29 ms, sys: 609 µs, total: 3.89 ms
Wall time: 3.34 ms


,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last,step,offset
name,,,,,,,,,
people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00,3,2
people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00,4,2
people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00,5,2
people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00,6,2
people0,Bakeries,merchantB,2020-04-30 14:19:00,70,offline,00:11:00,01:25:00,7,2


In [18]:
%%time
# columns = ["mcc", "transaction_value", "time_interval_to_next", "time_interval_from_last"]
test_data_query_route.drop(columns = ["offset"], inplace= True)
shaped_routes = test_data_query_route.pivot(index = test_data_query_route.index, columns =  "step")
shaped_routes.head()

CPU times: user 26.7 ms, sys: 1.49 ms, total: 28.2 ms
Wall time: 26.9 ms


mcc                                                          \
step         1    2                                                  3    
name                                                                      
people0     NaN  NaN  Motor Freight Carriers, Moving and Storage Com...   
people1     NaN  NaN                                                NaN   
people1002  NaN  NaN                                                NaN   
people1006  NaN  NaN                                                NaN   
people1012  NaN  NaN                                                NaN   

                                                               \
step                                                       4    
name                                                            
people0     Motor Freight Carriers, Moving and Storage Com...   
people1                                                   NaN   
people1002                                                NaN   
people1006                                                NaN   
people1012                                                NaN   

                                                                              \
step                                5                           6         7    
name                                                                           
people0     Drug Stores and Pharmacies  Drug Stores and Pharmacies  Bakeries   
people1          Fast Food Restaurants  Drug Stores and Pharmacies  Bakeries   
people1002                         NaN  Drug Stores and Pharmacies  Bakeries   
people1006                         NaN                         NaN  Bakeries   
people1012                         NaN                         NaN  Bakeries   

                                                               \
step                                                       8    
name                                                            
people0     Furniture, Home Furnishings, and Equipment Sto...   
people1     Furniture, Home Furnishings, and Equipment Sto...   
people1002                              Fast Food Restaurants   
people1006  Furniture, Home Furnishings, and Equipment Sto...   
people1012                         Drug Stores and Pharmacies   

                                        \
step                                9    
name                                     
people0          Fast Food Restaurants   
people1                       Bakeries   
people1002                 Book Stores   
people1006  Drug Stores and Pharmacies   
people1012  Drug Stores and Pharmacies   

                                                               ...  \
step                                                       10  ...   
name                                                           ...   
people0                                                   NaN  ...   
people1     Furniture, Home Furnishings, and Equipment Sto...  ...   
people1002                                                NaN  ...   
people1006  Motor Freight Carriers, Moving and Storage Com...  ...   
people1012                         Drug Stores and Pharmacies  ...   

           time_interval_from_last                                      \
step                            4        5        6        7        8    
name                                                                     
people0                   01:28:00 00:56:00 00:18:00 01:25:00 00:11:00   
people1                        NaT 01:58:00 02:07:00 02:00:00 00:25:00   
people1002                     NaT      NaT 03:16:00 02:12:00 00:26:00   
people1006                     NaT      NaT      NaT 03:08:00 02:15:00   
people1012                     NaT      NaT      NaT 02:26:00 01:37:00   

                                               
step             9        10       11  12  13  
name                                           
people0    02:13:00      NaT      NaT NaT NaT  
people1    00:37:00 01:28:00      NaT NaT NaT

In [19]:
%%time
conditioned_query_merchant = "merchantA"
filtered_routes.xs(("merchant", target_step), axis=1) == conditioned_query_merchant

NameError: name 'filtered_routes' is not defined

In [20]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(100)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = 1
#calculate step pos in the data
raw_step = single_sequence + left_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 5


if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
elif single_sequence < 0:
    source_step = raw_step+1
    target_step = raw_step
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

#filter links into last step routes, then all the sources should be same as last steps' target
if abs(single_sequence) > 1:
    links = links[links.index.isin(this_step_routes)]
    
# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
# get nodes grouped
nodes_g = links.drop(columns = ["source"]).groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links.drop(columns = ["source"]).groupby(["target"]).groups.values()
# get top nodes as output result
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# avoid 0 count nodes
nodes_result = nodes_result[nodes_result["count"] > 0]
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())

CPU times: user 42.9 ms, sys: 2.11 ms, total: 45 ms
Wall time: 43.2 ms


In [24]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(200)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = 1
#calculate step pos in the data
raw_step = single_sequence + left_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 5


if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
elif single_sequence < 0:
    source_step = raw_step+1
    target_step = raw_step
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

#filter links into last step routes, then all the sources should be same as last steps' target
if abs(single_sequence) > 1:
    links = links[links.index.isin(this_step_routes)]
    
# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
links["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
links["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)

# get nodes grouped
nodes_g = links[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_type":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links.groupby(["target"]).groups.values()
# filter: node count >0
nodes_g = nodes_g[nodes_g["count"] > 0].reset_index(drop=True)
# group to deal with transaction type
nodes_transtype_groupby = links[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_transtype = nodes_transtype_groupby.count().dropna()
# get routes from diff transaction types
nodes_transtype["type_routes"] = nodes_transtype_groupby.groups.values()
nodes_transtype_cnt = nodes_transtype.unstack()["transaction_value"].reset_index(drop=True)
nodes_transtype_routes = nodes_transtype.unstack()["type_routes"].reset_index(drop=True)
# add trans type count
nodes_g["offline_count"] = nodes_transtype_cnt["offline"]
nodes_g["online_count"] = nodes_transtype_cnt["online"]
# add trans type trajectory id
nodes_g["offline_route"] = nodes_transtype_routes["offline"]
nodes_g["online_route"] = nodes_transtype_routes["online"]
#add atv
nodes_g["atv"] = links.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get largest X results
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.reset_index().groupby(["source","target"]).count().dropna().rename(columns = {"name":"count"}).reset_index()
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
links_transtype_groupby = links[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_transtype = links_transtype_groupby.count().dropna()
# group to deal with transaction type
links_transtype["type_routes"] = links_transtype_groupby.groups.values()
links_transtype_cnt = links_transtype.unstack()["transaction_value"].reset_index(drop=True)
links_transtype_routes = links_transtype.unstack()["type_routes"].reset_index(drop=True)
# add trans type count
links_g["offline_count"] = links_transtype_cnt["offline"]
links_g["online_count"] = links_transtype_cnt["online"]
# add trans type traj id
links_g["offline_route"] = links_transtype_routes["offline"]
links_g["online_route"] = links_transtype_routes["online"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())

KeyError: 'count'

In [29]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(200)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = 1
#calculate step pos in the data
raw_step = single_sequence + left_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 5

CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 18.1 µs


In [42]:
# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
links["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
links["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)
# column lists that use later
columns_transaction_type = ["source", "target", "transaction_type", "transaction_value"]
columns_transaction_value = ["target", "transaction_value"]
# get nodes grouped
nodes_g = links[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links.groupby(["target"]).groups.values()
# filter: node count >0
nodes_g = nodes_g[nodes_g["count"] > 0].reset_index(drop=True)
# group to deal with transaction type
nodes_transtype_groupby = links[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_transtype = nodes_transtype_groupby.count().dropna()
# get routes from diff transaction types
nodes_transtype["type_routes"] = nodes_transtype_groupby.groups.values()
nodes_transtype_cnt = nodes_transtype.unstack()["transaction_value"].reset_index(drop=True)
nodes_transtype_routes = nodes_transtype.unstack()["type_routes"].reset_index(drop=True)
# add trans type count
nodes_g["offline_count"] = nodes_transtype_cnt["offline"]
nodes_g["online_count"] = nodes_transtype_cnt["online"]
# add trans type trajectory id
nodes_g["offline_route"] = nodes_transtype_routes["offline"]
nodes_g["online_route"] = nodes_transtype_routes["online"]
#add atv
nodes_g["atv"] = links.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get largest X results
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
nodes_result["offline_route"] = nodes_result["offline_route"].apply(lambda x: x.tolist())
nodes_result["online_route"] = nodes_result["online_route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.reset_index().groupby(["source","target"]).count().dropna().rename(columns = {"name":"count"}).reset_index()
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links[["source", "target", "transaction_value"]].groupby(["source","target"]).mean().dropna().reset_index()["transaction_value"]
links_transtype_groupby = links[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_transtype = links_transtype_groupby.count().dropna()
# group to deal with transaction type
links_transtype["type_routes"] = links_transtype_groupby.groups.values()
links_transtype_cnt = links_transtype.unstack()["transaction_value"].reset_index(drop=True)
links_transtype_routes = links_transtype.unstack()["type_routes"].reset_index(drop=True)
# add trans type count
links_g["offline_count"] = links_transtype_cnt["offline"]
links_g["online_count"] = links_transtype_cnt["online"]
# add trans type traj id
links_g["offline_route"] = links_transtype_routes["offline"]
links_g["online_route"] = links_transtype_routes["online"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())
links_result["offline_route"] = links_result["offline_route"].apply(lambda x: x.tolist())
links_result["online_route"] = links_result["online_route"].apply(lambda x: x.tolist())

In [43]:
links_result

step,source,target,count,transaction_type,merchant,route,atv,offline_count,online_count,offline_route,online_route
0,Bakeries,Bakeries,71.0,71.0,71.0,"[people1025, people1038, people1040, people108...",41.225352,31.0,40.0,"[people1179, people1182, people1207, people121...","[people1025, people1038, people1040, people108..."
1,Bakeries,Drug Stores and Pharmacies,319.0,319.0,319.0,"[people1012, people1014, people1017, people101...",43.946708,162.0,157.0,"[people1012, people1017, people1019, people102...","[people1014, people1024, people1036, people106..."
2,Bakeries,Fast Food Restaurants,137.0,137.0,137.0,"[people1002, people1015, people1021, people103...",45.116788,71.0,66.0,"[people1021, people1032, people1089, people115...","[people1002, people1015, people1060, people106..."
3,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",81.0,81.0,81.0,"[people0, people1, people1006, people1037, peo...",45.913580,35.0,46.0,"[people1037, people1197, people1217, people126...","[people0, people1, people1006, people1099, peo..."
4,Bakeries,"Grocery Stores, Supermarkets",58.0,58.0,58.0,"[people1016, people1049, people1093, people11,...",41.086207,31.0,27.0,"[people1093, people1116, people1151, people130...","[people1016, people1049, people11, people1203,..."


In [26]:
nodes_result

step,target,count,route,offline_count,online_count,offline_route,online_route,atv
0,Drug Stores and Pharmacies,319,"[people1012, people1014, people1017, people101...",162,157,"[people1012, people1017, people1019, people102...","[people1014, people1024, people1036, people106...",43.946708
1,Fast Food Restaurants,137,"[people1002, people1015, people1021, people103...",71,66,"[people1021, people1032, people1089, people115...","[people1002, people1015, people1060, people106...",45.116788
2,"Furniture, Home Furnishings, and Equipment Sto...",81,"[people0, people1, people1006, people1037, peo...",35,46,"[people1037, people1197, people1217, people126...","[people0, people1, people1006, people1099, peo...",45.913580
3,Bakeries,71,"[people1025, people1038, people1040, people108...",31,40,"[people1179, people1182, people1207, people121...","[people1025, people1038, people1040, people108...",41.225352
4,"Grocery Stores, Supermarkets",58,"[people1016, people1049, people1093, people11,...",31,27,"[people1093, people1116, people1151, people130...","[people1016, people1049, people11, people1203,...",41.086207


In [139]:
%%time
links_cpy = links.copy()
links_cpy["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
links_cpy["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)
links_cpy.head()

CPU times: user 3.32 ms, sys: 941 µs, total: 4.26 ms
Wall time: 3.4 ms


step,source,target,transaction_value,transaction_type,merchant
name,,,,,
people0,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",74.0,online,merchantB
people1,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",77.0,online,merchantA
people1002,Bakeries,Fast Food Restaurants,34.0,online,merchantC
people1006,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",37.0,online,merchantC
people1012,Bakeries,Drug Stores and Pharmacies,41.0,offline,merchantB


In [23]:
columns_transaction_type = ["source", "target", "transaction_type", "transaction_value"]
columns_transaction_value = ["target", "transaction_value"]
columns_merchant = ["target", "transaction_type", "merchant"]

In [22]:
%%time

# get nodes grouped
nodes_cpy_g = links_cpy[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_type":"count"}).reset_index()

# capture nodes' route
# nodes_cpy_g.sort_values(by = "count", ascending = False, inplace=True)
nodes_cpy_g["route"] = links_cpy.groupby(["target"]).groups.values()
nodes_cpy_g = nodes_cpy_g[nodes_cpy_g["count"] > 0].reset_index(drop=True)
nodes_transtype_groupby = links_cpy[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_cpy_transtype = nodes_transtype_groupby.count().dropna()
nodes_cpy_transtype["type_routes"] = nodes_transtype_groupby.groups.values()
nodes_cpy_transtype_cnt = nodes_cpy_transtype.unstack()["transaction_value"].reset_index(drop=True)
nodes_cpy_transtype_routes = nodes_cpy_transtype.unstack()["type_routes"].reset_index(drop=True)
nodes_cpy_g["offline_count"] = nodes_cpy_transtype_cnt["offline"]
nodes_cpy_g["online_count"] = nodes_cpy_transtype_cnt["online"]
nodes_cpy_g["offline_route"] = nodes_cpy_transtype_routes["offline"]
nodes_cpy_g["online_route"] = nodes_cpy_transtype_routes["online"]
nodes_cpy_g["atv"] = links_cpy.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get links grouped into source + target + count format df
links_cpy_g = links_cpy.reset_index().groupby(["source","target"]).count().dropna().rename(columns = {"name":"count"}).reset_index()
# capture links' routes by people's identifier/name
links_cpy_g["route"] = links_cpy.groupby(["source","target"]).groups.values()
# add atv column
links_cpy_g["atv"] = links_cpy.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
links_transtype_groupby = links_cpy[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_cpy_transtype = links_transtype_groupby.count().dropna()
links_cpy_transtype["type_routes"] = links_transtype_groupby.groups.values()
links_cpy_transtype_cnt = links_cpy_transtype.unstack()["transaction_value"].reset_index(drop=True)
links_cpy_transtype_routes = links_cpy_transtype.unstack()["type_routes"].reset_index(drop=True)
# links_cpy_transtype["type_routes"] = 
links_cpy_g["offline_count"] = links_cpy_transtype_cnt["offline"]
links_cpy_g["online_count"] = links_cpy_transtype_cnt["online"]
links_cpy_g["offline_route"] = links_cpy_transtype_routes["offline"]
links_cpy_g["online_route"] = links_cpy_transtype_routes["online"]
# links_cpy.groupby(["source","target"]).mean()
# link_cpy_transtype_cnt.reset_index(drop=True)
nodes_cpy_g.nlargest(keep_rank_num+5, "count").reset_index(drop=True)
links_cpy_g

NameError: name 'links_cpy' is not defined

In [259]:
links_cpy[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"]).count().unstack()["transaction_value"].dropna().reset_index(drop=True)["offline"]

0     31.0
1    162.0
2     71.0
3     35.0
4     31.0
Name: offline, dtype: float64

In [150]:
%%time

# nodes_copy = links_copy.drop(columns = ["source"])
# links_copy["target"].value_counts()
# # get links grouped into source + target + count format df
# links_cpy_g = links_cpy.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
# # capture links' routes by people's identifier/name
# links_cpy_g["route"] = links_cpy.groupby(["source","target"]).groups.values()
# # add atv column
# links_cpy_g["atv"] = links_cpy.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]

links_cpy[columns_transaction_type].groupby(by = ["target", "transaction_type"])["transaction_type"].count().unstack()[["offline"]]

CPU times: user 11.9 ms, sys: 1.84 ms, total: 13.7 ms
Wall time: 12.1 ms


transaction_type,offline
target,
Bakeries,31
Book Stores,0
Drug Stores and Pharmacies,162
Fast Food Restaurants,71
"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",35
"Grocery Stores, Supermarkets",31
"Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local",0


In [85]:
nodes_result

step,target,count,route
0,Drug Stores and Pharmacies,191,"[people1012, people1014, people1019, people102..."
1,Fast Food Restaurants,75,"[people1002, people1021, people1032, people106..."
2,Bakeries,41,"[people1025, people1038, people1040, people108..."
3,"Furniture, Home Furnishings, and Equipment Sto...",41,"[people0, people1, people1037, people1099, peo..."
4,"Grocery Stores, Supermarkets",32,"[people1049, people1093, people11, people1151,..."


In [67]:
%%time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
time_interval_limit = pd.offsets.Minute(100)
sample_last_step_list = ["Bakeries", "Book Stores"]
QuerySingleNode(startTime, endTime, "Bakeries", -2, time_interval_limit, sample_last_step_list, 5)

CPU times: user 96.9 ms, sys: 2.49 ms, total: 99.4 ms
Wall time: 98.4 ms


step,source,target,count,route,atv
0,Book Stores,Fast Food Restaurants,11.0,"[people2587, people3052, people3135, people377...",39.363636
1,Book Stores,Book Stores,6.0,"[people1616, people3649, people4895, people513...",43.833333
2,Book Stores,"Motor Freight Carriers, Moving and Storage Com...",5.0,"[people2132, people3056, people342, people384,...",35.800000
3,Book Stores,Drug Stores and Pharmacies,1.0,[people6197],46.000000
4,Book Stores,"Grocery Stores, Supermarkets",1.0,[people1427],55.000000
